In [1]:
import os,sys
import shutil,glob
import pickle
import path
from tqdm import tqdm
import hashlib

In [2]:
rearrange_dir = '../../RAID6/original_tmp_data/0223-0307/'

In [3]:
all_files = glob.glob(rearrange_dir+'**/*.tar.gz', recursive=True)
all_files

['../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_23/pcap_2020_02_23.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_24/pcap_2020_02_24_part3.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_24/pcap_2020_02_24_part4.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_25/pcap_2020_02_25_part1.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_25/pcap_2020_02_25_part2.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_25/pcap_2020_02_25_part3.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_25/pcap_2020_02_25_part4.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_26/pcap_2020_02_26_part1.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_26/pcap_2020_02_26_part2.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_26/pcap_2020_02_26_part3.tar.gz',
 '../../RAID6/original_tmp_data/0223-0307/中嘉寬頻/pcap/2020_02_26/pca

.split('/') :
* 4=>pcap/hp
* 3=>ISP (轉換)
* 5=>date (注意格式)
* 6=>fileName
***
#### 最原本.tar.gz/.json TTC所給之原始檔案目前位於RAID5目錄下
* pcap 
    * date
        * ISP
            * fileName
* honeypot 
    * date
        * type
            * ISP or mongoDB
                * fileName

#### 經解壓縮、加工處理後等大size個別資料目前位於 data_hdd4t目錄下
* pcap_process
    * date
        * ISP
            * fileName
* honeypot_process
    * date
        * type
            * ISP or mongoDB
                * fileName

#### spark dataframe、pickle等經過整合或小size資料目前位於spark_data目錄下
* pcap_inter
    * date
        * ISP
            * fileName
* honeypot_inter
    * date
        * type
            * ISP or mongoDB
                * fileName

In [7]:
def aliase_convert(s):
    '''
    dictionary baesd conversion
    '''
    if s == '台哥大':
        return '台灣大哥大'
    elif s == '台基開發':
        return '台灣基礎開發'
    elif s == '亞太':
        return '亞太電信'
    elif s == '遠傳':
        return '遠傳電信'
    else:
        return s

In [9]:
idx = -1
print(all_files[idx].split('/'))
len(all_files[idx].split('/'))
#10=>hp
#9=>pcap

['..', '..', 'RAID6', 'original_tmp_data', '0223-0307', '遠傳', 'honeypot', 'cowrie', '2020_03_07', 'cowrie_log_2020_03_07.tar.gz']


10

In [8]:
for i,path in (enumerate(all_files)):
    if 'mongo' in path:
        print(i) #不應該要有印出東西
        break

In [10]:
for path in tqdm(all_files):
    path_li = path.split('/')
    if path_li[3] in ['pcap','honeypot','metadata']: #如果是已經分好在RAID5的就不要動 #需要改
        continue
    if len(path_li) == 9: #pcap #需要改
        date_dir = path_li[-2] #日期須修改
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        isp_dir = path_li[-4] #isp名稱位置 #需要改
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert 'pcap' in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = '../../RAID5/'+'pcap/'+date_dir+'/'+isp_dir #要儲存的位置 #需要改
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)
        try:
            shutil.move(path,save_dir+'/')
        except :
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open('../../RAID5/'+'pcap/'+date_dir+'/'+isp_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest:
                os.remove(path)
            else:
                print('_2 was created')                
                shutil.move(path,save_dir+'/'+fileName+'_2')
    elif len(path_li) == 10: #hp #長度需要改
        date_dir = path_li[-2] #日期須檢查
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        type_dir = path_li[-3] #hp的type #需要改
        assert type_dir in ['amun', 'cowrie', 'dionaea',  'glastopf']
        isp_dir = path_li[-5] #isp名稱位置 #需要改
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert type_dir in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = '../../RAID5/'+'honeypot/'+date_dir+'/'+type_dir+'/'+isp_dir#要儲存的位置 #需要改
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)        
        try:
            shutil.move(path,save_dir+'/')
        except : #重複一模一樣的檔名
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open(save_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest: #真的一模一樣舊移除舊的
                os.remove(path)
            else: #不一樣舊都保留到新的位置去
                print('_2 was created')
                shutil.move(path,save_dir+'/'+fileName+'_2')
    else:
        print("WARNING!!Assertion failed, not pcap nor hp. :",path)
            

100%|██████████| 1773/1773 [24:32:26<00:00, 49.83s/it]    


In [11]:
def recursive_delete_if_empty(path):
    """Recursively delete empty directories; return True
    if everything was deleted."""

    if not os.path.isdir(path):
        # If you also want to delete some files like desktop.ini, check
        # for that here, and return True if you delete them.
        return False

    # Note that the list comprehension here is necessary, a
    # generator expression would shortcut and we don't want that!
    if all([recursive_delete_if_empty(os.path.join(path, filename))
            for filename in os.listdir(path)]):
        # Either there was nothing here or it was all deleted
        os.rmdir(path)
        return True
    else:
        return False

In [12]:
one = recursive_delete_if_empty(rearrange_dir)
# two = recursive_delete_if_empty(rearrange_dir+'0126_0208')
# three = recursive_delete_if_empty(rearrange_dir+'1229_0112')
print(one)#,two,three)

False


### Mongodb log from  Mybook
* 須先把rm -rf System\ Volume\ Information/ \$RECYCLE.BIN/

In [32]:
src_dir = rearrange_dir#'../RAID5/Mongodblog/'

In [33]:
all_paths = glob.glob(src_dir+'**/*.json', recursive=True)
print(len(all_paths))
all_paths

7248


['../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-18.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-17.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-16.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-15.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-14.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-13.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-12.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-11.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-10.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-04-09.json',
 '../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.f

In [34]:
all_paths[-1]

'../../RAID5/original_tmp_data/0405-0418/mongo/台灣基礎開發/amun/amun.events_2019-12-29.json'

In [ ]:
for path in tqdm(all_paths):
    path_li = path.split('/')
    #以下要隨路徑而改
    date_dir = path_li[-1].split('_')[-1].split('.')[0]
    date_dir = date_dir.replace('-','_')
    assert len(date_dir.split('_')) == 3
    type_dir = path_li[-2]
    assert type_dir in ['amun', 'cowrie', 'dionaea',  'glastopf']
    isp_dir = path_li[-3]
    isp_dir = aliase_convert(isp_dir)
    fileName = path_li[-1]
    assert type_dir in fileName
    assert date_dir.split('_')[-1] in fileName
    assert '.json' in fileName
    save_dir = '../../RAID5/'+'honeypot/'+date_dir+'/'+type_dir+'/mongoDB/'+isp_dir #要儲存的位置 #需要改
    if not os.path.exists(save_dir):
        os.makedirs(save_dir,exist_ok=True)        
    try:
        shutil.move(path,save_dir+'/')
    except : #重複一模一樣的檔名
        print(path)
        ori = hashlib.md5(open(path,'rb').read()).hexdigest()
        dest =  hashlib.md5(open(save_dir+'/'+fileName,'rb').read()).hexdigest()
        if ori == dest: #真的一模一樣舊移除舊的
            pass
            os.remove(path)
        else: #不一樣舊都保留到新的位置去
            print('_2 was created')
            shutil.move(path,save_dir+'/'+fileName+'_2')    

  1%|▏         | 98/7248 [00:03<02:48, 42.53it/s]

../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-11.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.events_2020-01-11.json


  3%|▎         | 198/7248 [00:03<02:01, 57.85it/s]

../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-10.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-09.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-08.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-07.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-06.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-05.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-04.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-03.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-02.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2020-01-01.json
../../RAID5/original_tmp_data/0405-0418/mongo/遠傳/glastopf/glastopf.files_2019-12-31.json
../../RAID5/original_

  3%|▎         | 212/7248 [00:03<02:22, 49.33it/s]

...

../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-10.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-09.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-08.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-07.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-06.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-05.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-04.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-03.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-02.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2020-01-01.json
../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2019

 22%|██▏       | 1599/7248 [13:06<14:34,  6.46it/s]

../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/glastopf/glastopf.events_2019-12-29.json


 23%|██▎       | 1696/7248 [13:07<01:33, 59.44it/s]

../../RAID5/original_tmp_data/0405-0418/mongo/亞太電信/dionaea/dionaea.connections_2020-01-11.json


In [39]:
recursive_delete_if_empty(src_dir)

True

## original old data

In [29]:
src_dir = '../data_hdd4t/中華電信/'

In [20]:
all_files = glob.glob(src_dir+'**/*.tar.gz', recursive=True)
print(len(all_files))
all_files

635


['../data_hdd4t/亞太/honeypot/2020-01-10/cowrie/cowrie_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/cowrie/cowrie_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/amun/amun_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/amun/amun_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/glastopf/glastopf_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/glastopf/glastopf_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/dionaea/dionaea_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/dionaea/dionaea_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/cowrie/cowrie_log_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/cowrie/cowrie_file_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/amun/amun_log_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/amun/amun_file_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/glastopf/glastopf_file_2020-01-11.t

In [24]:
for path in tqdm(all_files):
    path_li = path.split('/')
    if path_li[2] in ['pcap','honeypot','metadata']:
        continue
    if len(path_li) == 5: #pcap
        date_dir = path_li[-1].split('.')[1]
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        isp_dir = path_li[2]
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert 'snort' in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = rearrange_dir+'pcap/'+date_dir+'/'+isp_dir#+'/'+fileName
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)
        try:
            shutil.move(path,save_dir+'/')
        except :
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open(rearrange_dir+'pcap/'+date_dir+'/'+isp_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest:
                os.remove(path)
            else:
                print('_2 was created')                
                shutil.move(path,save_dir+'/'+fileName+'_2')
    elif len(path_li) == 7: #hp
        date_dir = path_li[-3]
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        type_dir = path_li[-2]
        assert type_dir in ['amun', 'cowrie', 'dionaea',  'glastopf']
        isp_dir = path_li[2]
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert type_dir in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = rearrange_dir+'honeypot/'+date_dir+'/'+type_dir+'/'+isp_dir#+'/'+fileName
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)        
        try:
            shutil.move(path,save_dir+'/')
        except : #重複一模一樣的檔名
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open(save_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest: #真的一模一樣舊移除舊的
                os.remove(path)
            else: #不一樣舊都保留到新的位置去
                print('_2 was created')
                shutil.move(path,save_dir+'/'+fileName+'_2')
            

 17%|█▋        | 106/635 [09:35<02:22,  3.72it/s]  

../data_hdd4t/台灣碩網/honeypot/2020-01-03/cowrie/cowrie_log_2020-01-03.tar.gz
../data_hdd4t/台灣碩網/honeypot/2020-01-03/amun/amun_file_2020-01-03.tar.gz


 17%|█▋        | 107/635 [09:36<02:09,  4.09it/s]

../data_hdd4t/台灣碩網/honeypot/2020-01-03/amun/amun_log_2020-01-03.tar.gz


 17%|█▋        | 108/635 [09:37<03:52,  2.27it/s]

...

 75%|███████▌  | 477/635 [4:01:22<04:52,  1.85s/it]

../data_hdd4t/中華電信/honeypot/2020-01-01/amun/amun_log_2020-01-01.tar.gz
../data_hdd4t/中華電信/honeypot/2020-01-01/glastopf/glastopf_file_2020-01-01.tar.gz
../data_hdd4t/中華電信/honeypot/2020-01-01/glastopf/glastopf_log_2020-01-01.tar.gz
../data_hdd4t/中華電信/honeypot/2020-01-01/dionaea/dionaea_log_2020-01-01.tar.gz


 76%|███████▌  | 481/635 [4:01:23<03:23,  1.32s/it]

../data_hdd4t/中華電信/honeypot/2020-01-01/dionaea/dionaea_file_2020-01-01.tar.gz


100%|██████████| 635/635 [7:07:56<00:00, 40.43s/it]    


In [28]:
recursive_delete_if_empty(src_dir)

False

In [27]:
src_dir

'../data_hdd4t/'

In [23]:
len('../data_hdd4t/台灣碩網/pcap/snort.2020-01-06.tar.gz'.split('/'))

5